`text2vec-contextionary` is available throguh **Weaviate open-source ONLY**. Meaning, you will need to configure a `docker-compose.yaml` file. 

Check out [Weaviate's configuration tool](https://weaviate.io/developers/weaviate/installation/docker-compose#configurator) to create one.

## Dependencies

In [ ]:
!pip install --pre -I "weaviate-client==4.3.b2"

## Connect to Weaviate

In [ ]:
import weaviate

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local()

client.is_ready()

## Create a collection
> Collection stores your data and vector embeddings.

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes as wvc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wvc.Configure.Vectorizer.text2vec_contextionary( # specify the vectorizer and model type you're using
        vectorize_class_name=True
    ),

    properties=[ # defining properties (data schema) is optional
        wvc.Property(name="Question", data_type=wvc.DataType.TEXT),
        wvc.Property(name="Answer", data_type=wvc.DataType.TEXT),
        wvc.Property(name="Category", data_type=wvc.DataType.TEXT, skip_vectorization=True),
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search on a specific property

The `properties` parameter allows you to list the properties that you want bm25 to search on.

In [ ]:
response = jeopardy.query.hybrid(
    query="northern beast",
    query_properties=["question"],
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [ ]:
import weaviate.classes as wvc # wvc is an alias to save us from typing weaviate.classes everywhere ;)

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    filters=wvc.Filter("category").equal("Animals"),
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a custom vector

You can pass in your own vector as input into the hybrid query, by using the `vector` parameter. 

In [ ]:
vector = [0.133922210335731506, -0.27315509766340256, 0.36337894797325134, 0.48631668388843536, -0.52181884944438934, 0.01682819239795208, -0.14903388917446136, -0.06788449734449387, 0.16681791841983795, 0.37949123978614807, 0.17640919983386993, 0.11782828718423843, 0.09980925172567368, -0.15210925042629242, 0.11096711456775665, 0.15886759757995605, -0.3046692907810211, -0.05466936156153679, -0.30536478757858276, -0.1749650090932846, -0.1273302286863327, 0.12550920248031616, -0.09845238924026489, 0.08963415026664734, -0.18756364285945892, -0.015974024310708046, 0.1620747148990631, 0.008896538987755775, -0.25837472081184387, 0.040347710251808167, -0.11426491290330887, 0.08805316686630249, 0.24871617555618286, 0.08696815371513367, -0.14896774291992188, -0.1394026130437851, -0.09163132309913635, 0.14358510076999664, -0.1584167331457138, 0.5675868988037109, -0.03471371531486511, 0.19687999784946442, 0.29458776116371155, 0.12294784188270569, 0.2704301178455353, 0.06848172098398209, 0.21694298088550568, 0.08867599815130234, -0.21569298207759857, -0.13814939558506012, -0.3671528398990631, 0.20951882004737854, 0.1414112150669098, -0.016151469200849533, -0.1869555562734604, -0.15464437007904053, 0.03912660479545593, -0.12775863707065582, 0.2579288184642792, -0.11487586796283722, 0.3893434405326843, -0.5490038990974426, -0.42180824279785156, 0.0014822649536654353, 0.10005516558885574, -0.009532607160508633, -0.1705951988697052, -0.2065221071243286, 0.23923087120056152, 0.12221403419971466, -0.3253549337387085, -0.1663360446691513, -0.19168059527873993, 0.025366833433508873, 0.3128953278064728, -0.34270963072776794, 0.22247213125228882, -0.29601749777793884, 0.003241761587560177, 0.03251703456044197, -0.08605765551328659, -0.5389830470085144, 0.09075367450714111, -0.0963517278432846, 0.39265695214271545, -0.1338977813720703, 0.21237137913703918, 0.16107212007045746, -0.010967296548187733, 0.1253993660211563, 0.03918381407856941, 0.10936978459358215, -0.10970871895551682, -0.22586868703365326, 0.011124550364911556, 0.16587188839912415, -0.09446170181035995, -0.22812534868717194, 0.055585213005542755, 0.5546377897262573, -0.04449738562107086, -0.3611614406108856, 0.6435457468032837, -0.17875909805297852, -0.31694623827934265, -0.242616206407547, 0.18302279710769653, -0.35096609592437744, -0.13490359485149384, -0.06443284451961517, 0.0034323460422456264, -0.09598598629236221, -0.13936910033226013, -0.08243658393621445, -0.3080992102622986, -0.2132837325334549, 0.06229309365153313, -0.3691481947898865, 0.014200489968061447, -0.02915835939347744, 0.17643195390701294, 0.1874920129776001, -0.148249551653862, 0.0347103551030159, -0.03744988143444061, -0.04782085120677948, 1.4108911752700806, 0.3615219295024872, -0.09007949382066727, 0.057988401502370834, 0.09386436641216278, -0.030105572193861008, -0.215859055519104, -0.650331974029541, 0.24405989050865173, 0.19997435808181763, 0.1976812183856964, 0.027195604518055916, 0.05475842207670212, -0.19449888169765472, 0.12570394575595856, -0.09395390748977661, -0.03743208199739456, -0.24666397273540497, 0.3717946410179138, 0.24101202189922333, -0.30562683939933777, 0.041510336101055145, 0.07934113591909409, 0.029812229797244072, 0.13256244361400604, 0.09144382923841476, -0.2194734662771225, 0.1050177738070488, 0.07906405627727509, 0.13477979600429535, 0.027775397524237633, 0.08713040500879288, -0.17142833769321442, 0.10657251626253128, -0.35243573784828186, -0.4155956208705902, 0.07885827869176865, 0.050501905381679535, -0.11574598401784897, 0.09363331645727158, 0.2174544781446457, -0.19112569093704224, -0.025532107800245285, -0.05494973063468933, -0.3588777482509613, 0.12829390168190002, -0.050472501665353775, 0.002819460118189454, -0.41528624296188354, -0.2325700968503952, -0.1750108301639557, -0.011163167655467987, 0.3948625922203064, 0.14674773812294006, -0.18696045875549316, -0.7558386921882629, 0.1775888353586197, -0.20077720284461975, 0.4181656837463379, 0.044551681727170944, 0.0140397222712636, 0.0621115118265152, -0.2208034098148346, 0.08778870105743408, 0.06158978119492531, 0.01304273959249258, 0.0778842344880104, 0.009456549771130085, -0.2646521329879761, 0.026772527024149895, -0.030432146042585373, 0.14172272384166718, -0.1257399171590805, -0.14118435978889465, -0.25049659609794617, 0.08980756998062134, -0.03352894261479378, -0.06505658477544785, -0.21219632029533386, 0.08123009651899338, 0.2491193413734436, -0.07238374650478363, -0.029995454475283623, -0.0007849896210245788, -0.22551272809505463, 0.10677415132522583, -0.060766853392124176, 0.5189664959907532, 0.010593662969768047, 0.23315128684043884, 0.26671102643013, 0.19512870907783508, -0.25564122200012207, -0.08399118483066559, 0.0819086879491806, 0.07381071895360947, 0.16994650661945343, 0.0326274037361145, 0.14035676419734955, 0.07520456612110138, -0.45695576071739197, 0.020487410947680473, 0.08721856772899628, -0.32976964116096497, -0.22933940589427948, -0.49831244349479675, -0.04490511491894722, 0.252360075712204, 0.09503661096096039, -0.01729903183877468, 0.12610195577144623, 0.25541263818740845, -0.18188682198524475, -0.03288526460528374, 0.190602645277977, -0.37432682514190674, 0.22453762590885162, 0.12411367148160934, 0.09773889183998108, 0.13075095415115356, -0.09447057545185089, 0.06994586437940598, 0.1663770228624344, 0.1969572752714157, -0.019351452589035034, 0.1037159338593483, -0.07371033728122711, 0.11163657903671265, -0.004621752537786961, 0.1239633709192276, -0.249355286359787, -0.14096909761428833, -0.3970746695995331, 0.03178446739912033, 0.0016118615167215466, 0.13274089992046356, 0.0983293354511261, -0.14118865132331848, 0.037634532898664474, -0.171011283993721, 0.112472303211689, 0.05132365599274635, -0.4928515553474426, -0.15134254097938538, 0.13785603642463684, -0.18253745138645172, 0.18178880214691162, -0.10481800884008408, -0.00135972595307976, -0.08431749790906906, -0.3371056914329529, 0.1059824600815773, 0.3133571743965149, 0.13559016585350037, 0.10154692083597183, -0.29814088344573975, 0.4135531485080719, 0.10298778116703033, 0.6066415905952454, 0.05883769690990448, -0.10505139082670212, 0.17205993831157684, -0.08872780203819275, 0.3165540397167206, -0.025006478652358055, 0.1396600902080536, 0.05534999445080757, -0.18008781969547272, -0.2792469561100006, 0.26451390981674194, 0.15396876633167267, -0.05094185099005699, 0.2226734608411789, -0.12221679836511612]

response = jeopardy.query.hybrid(
    query="animal",
    vector=vector,
    limit=2
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")